In [1]:
import emissor
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis

/Users/piek/Desktop/t-MA-Combots-2021/code/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import os
import time
import uuid
from datetime import datetime
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType
from datetime import datetime
from datetime import date
import requests

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util

In [3]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
agent = "Leolani2"
human = "Stranger"

### The name of your scenario
scenarioid = "myscenario3"

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "./data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenarioid + "/" + "image"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenarioid)
scenario = scenarioStorage.create_scenario(scenarioid, datetime.now().microsecond, datetime.now().microsecond, agent)

Directory  ./data/myscenario3  already exists
Directory  ./data/myscenario3/image  already exists


In [4]:
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)

<class 'pathlib.PosixPath'>
2021-10-27 17:20:57,209 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-10-27 17:20:57,210 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Clearing brain
2021-10-27 17:20:59,371 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Checking if ontology is in brain
2021-10-27 17:20:59,373 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Posting query
2021-10-27 17:20:59,849 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-10-27 17:21:02,073 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-10-27 17:21:02,075 -    DEBUG -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-10-27 17:21:02,078 -    DEBUG -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-10-27 17:21:02,079 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-10-27 17:21:02,323 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query
2021-10-27 17:21:02,368 -     INF

In [ ]:
#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = "Hi there. Who are you " + human + "?"
print(agent + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = input('\n')
print(human + ": " + utterance)
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    chat = Chat(human)
    chat.add_utterance([UtteranceHypothesis(utterance, 1.0)])
    chat.last_utterance.analyze()
    triple = rephrase_triple_json_for_capsule(chat.last_utterance.triple)
    ## Post triples to the brain:

        # No triple was extracted, so we missed three items (s, p, o)
    if chat.last_utterance.triple is None:
        utterance = "Any gossip" + '\n'
    else:
        print
        # A triple was extracted so we compare it elementwise
        capsule = c_util.scenario_utterance_and_triple_to_capsule(scenario, 
                                                                  place_id,
                                                                  location,
                                                                  textSignal, 
                                                                  human,
                                                                  chat.last_utterance.perspective, 
                                                                  triple)
        print('Capsule:', capsule)
        response = my_brain.update(capsule, reason_types=True)
        #print(thoughts)
    
    # Create the response from the system and store this as a new signal
    utterance = "" #ttt.getTextFromTriples(response)
    if not utterance:
            utterance = "So you what do you want to talk about?\n"

    response = utterance[::-1]
    print(agent + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    ###### Getting the next input signals
    utterance = input('\n')


Leolani2: Hi there. Who are you Stranger?



 I am Piek


Stranger: I am Piek
2021-10-27 17:25:02,496 -     INFO - cltl.triple_extraction.api.Chat (Stranger)          000 - << Start of Chat with Stranger >>
2021-10-27 17:25:03,419 -     INFO - cltl.triple_extraction.api.Chat (Stranger)          001 -   Stranger: "I am Piek"
{'subject': {'label': 'Stranger', 'type': ['noun.person']}, 'predicate': {'label': 'be', 'type': ['verb.stative']}, 'object': {'label': 'Piek', 'type': ['person']}}
{'subject': {'person', 'Stranger'}, 'predicate': {'be', 'stative'}, 'object': {'person', 'Piek'}}
Capsule: {'chat': 'myscenario3', 'turn': '7278ef19-b75c-45f6-aa89-1db7f5a56520', 'author': 'carl', 'utterance': 'I am Piek', 'utterance_type': <UtteranceType.STATEMENT: 0>, 'position': '0-9', 'subject': {'label': 'piek', 'type': 'person'}, 'predicate': {'type': 'see'}, 'object': {'label': 'pills', 'type': 'object'}, 'perspective': {'sentiment': 0, 'certainty': 1, 'polarity': 1, 'emotion': <Emotion.NEUTRAL: 7>}, 'context_id': 'Leolani2', 'date': datetime.date(2021, 